In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import tensorflow as tf
from numpy import *
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tensorflow.keras import layers,datasets
from sklearn.preprocessing import LabelEncoder,MinMaxScaler,StandardScaler
from tensorflow import keras
import gc
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
import category_encoders as ce
from sklearn.metrics import f1_score,classification_report,roc_auc_score

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
plan_info = pd.read_csv('./ptom_third_plan.csv')
plan_info.dropna(how='all', inplace=True, axis=1)
plan_info.drop(['ad_start_time', 'ad_end_time', 'remark','origin'], axis=1, inplace=True)
plan_info.dropna(subset=['ad_info'],inplace=True)
plan_info.drop(['ad_id', 'ad_group_id', 'ad_name','media_id','plan_id','launch_op_id','bid_mode','bid_status','bid',
                'budget_mode','budget','opt_status','status','syn_time','update_time','inventory_type'], axis=1, inplace=True)
# 解析json
plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
temp = plan_info['ad_info'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('ad_info', axis=1, inplace=True)
temp = plan_info['audience'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('audience', axis=1, inplace=True)
temp = plan_info['action'].apply(pd.Series)
plan_info = pd.concat([plan_info,temp], axis=1)
plan_info.drop('action', axis=1, inplace=True)
plan_info.dropna(how='all', inplace=True, axis=1)

plan_info_2 = plan_info.copy()
plan_info_2.drop(['game_package_batch_id','advanced_creative_type','game_package_desc','game_package_thumbnail_ids',
               'external_url','intelligent_flow_switch','audience_package_id','union_video_type','converted_time_duration',
               'roi_goal','device_type','deep_bid_type','auto_extend_targets'],axis=1,inplace=True)
plan_info_2.drop(['package','open_url','ad_modify_time','deep_cpabid','download_url','download_mode','id','campaign_id',
               'package','ad_create_time','start_time','ad_id','name','status','modify_time','advertiser_id','schedule_type','quick_app_url',
               'end_time','convert_id','audit_reject_reason','download_type','budget_mode','bid','pricing',
               'include_custom_actions','app_type','exclude_flow_package','aweme_fan_behaviors','business_ids','user_type',
               'activate_type','aweme_fan_accounts','aweme_fan_categories','aweme_fans_numbers','article_category',
                'flow_package','carrier','superior_popularity_type','device_brand','exclude_custom_actions','geolocation',
               'opt_status','learning_phase','district'],axis=1,inplace=True)
# 删除老版兴趣行为定向
plan_info_2.drop(['ad_tag','interest_tags','app_behavior_target','app_category','app_ids'],axis=1,inplace=True)
# 删除老版兴趣行为定向
plan_info_2.drop(['retargeting_tags'],axis=1,inplace=True)

In [4]:
image_info = pd.read_csv('./image_info.csv')
df = pd.merge(plan_info_2,image_info,on=['channel_id','source_id'],how='left')
df.dropna(subset=['image_id'],inplace=True)

In [5]:
df.rename(columns={'create_time_x':'create_time','create_time_y':'create_date'},inplace=True)
df = df[['ad_account_id', 'game_id', 'channel_id', 'source_id', 'create_time','create_date',
         'bid2','budget','inventory_type', 'schedule_time', 'delivery_range','flow_control_mode',
        'smart_bid_type', 'adjust_cpa', 'cpa_bid', 'hide_if_exists', 'hide_if_converted',
        'auto_extend_enabled', 'age','gender', 'city', 'location_type', 'platform', 'launch_price','retargeting_type',
         'retargeting_tags_include','retargeting_tags_exclude', 'ac', 'android_osv','ios_osv', 
        'interest_action_mode', 'interest_categories', 'interest_words',
       'action_categories', 'action_days', 'action_scene', 'action_words',
       'image_id', 'label_ids']]

In [6]:
launch_report = pd.read_csv('./launch_report.csv',usecols=['tdate','channel_id', 'source_id', 'amount',
       'create_role_num', 'pay_role_user_num', 'new_role_money'])
launch_report.sort_values(by='tdate',inplace=True)
launch_report.drop_duplicates(subset=['channel_id','source_id'],keep='first',inplace=True)
df = pd.merge(df, launch_report,on=['channel_id','source_id'],how='left')
df.drop(df[df['tdate'].isna()].index,inplace=True)
df['label'] = ((df['pay_role_user_num']>0)&(df['amount']>=500)).astype(int)
df['image_id'] = df['image_id'].astype('int')
df['create_time'] = pd.to_datetime(df['create_time'])

In [7]:
# 读取image不同窗口期的运营数据
image_train_data_1 = pd.read_csv('./image_data/train_data_1.csv')
image_train_data_1['im_create_role_pay_rate_1'] = image_train_data_1.apply(lambda x:0 if x.im_create_role_num_1==0 else 
                                                            x.im_pay_role_user_num_1 / x.im_create_role_num_1,axis=1)
image_train_data_1['im_create_role_cost_1'] = image_train_data_1.apply(lambda x:np.inf if x.im_create_role_num_1==0 else 
                                                            x.im_amount_1 / x.im_create_role_num_1,axis=1)
image_train_data_1['im_create_role_pay_cost_1'] = image_train_data_1.apply(lambda x:np.inf if x.im_pay_role_user_num_1==0 else 
                                                            x.im_amount_1 / x.im_pay_role_user_num_1,axis=1)
image_train_data_1['im_valid_source_rate_1'] = image_train_data_1.apply(lambda x:0 if x.im_source_num_1==0 else 
                                                            x.im_valid_source_num_1 / x.im_source_num_1,axis=1)
image_train_data_1.drop(['data_win_1','label_ids_1'],axis=1,inplace=True)
image_train_data_1.rename(columns={'model_run_datetime_1':'create_date','image_id_1':'image_id'},inplace=True)
df = pd.merge(df, image_train_data_1, on=['image_id','create_date'],how='left', validate='many_to_one')

image_train_data_3 = pd.read_csv('./image_data/train_data_3.csv')
image_train_data_3['im_create_role_pay_rate_3'] = image_train_data_3.apply(lambda x:0 if x.im_create_role_num_3==0 else 
                                                            x.im_pay_role_user_num_3 / x.im_create_role_num_3,axis=1)
image_train_data_3['im_create_role_cost_3'] = image_train_data_3.apply(lambda x:np.inf if x.im_create_role_num_3==0 else 
                                                            x.im_amount_3 / x.im_create_role_num_3,axis=1)
image_train_data_3['im_create_role_pay_cost_3'] = image_train_data_3.apply(lambda x:np.inf if x.im_pay_role_user_num_3==0 else 
                                                            x.im_amount_3 / x.im_pay_role_user_num_3,axis=1)
image_train_data_3['im_valid_source_rate_3'] = image_train_data_3.apply(lambda x:0 if x.im_source_num_3==0 else 
                                                            x.im_valid_source_num_3 / x.im_source_num_3,axis=1)
image_train_data_3.drop(['data_win_3','label_ids_3'],axis=1,inplace=True)
image_train_data_3.rename(columns={'model_run_datetime_3':'create_date','image_id_3':'image_id'},inplace=True)
df = pd.merge(df, image_train_data_3, on=['image_id','create_date'],how='left', validate='many_to_one')

image_train_data_7 = pd.read_csv('./image_data/train_data_7.csv')
image_train_data_7['im_create_role_pay_rate_7'] = image_train_data_7.apply(lambda x:0 if x.im_create_role_num_7==0 else 
                                                            x.im_pay_role_user_num_7 / x.im_create_role_num_7,axis=1)
image_train_data_7['im_create_role_cost_7'] = image_train_data_7.apply(lambda x:np.inf if x.im_create_role_num_7==0 else 
                                                            x.im_amount_7 / x.im_create_role_num_7,axis=1)

image_train_data_7['im_create_role_pay_cost_7'] = image_train_data_7.apply(lambda x:np.inf if x.im_pay_role_user_num_7==0 else 
                                                            x.im_amount_7 / x.im_pay_role_user_num_7,axis=1)
image_train_data_7['im_valid_source_rate_7'] = image_train_data_7.apply(lambda x:0 if x.im_source_num_7==0 else 
                                                            x.im_valid_source_num_7 / x.im_source_num_7,axis=1)
image_train_data_7.drop(['data_win_7','label_ids_7'],axis=1,inplace=True)
image_train_data_7.rename(columns={'model_run_datetime_7':'create_date','image_id_7':'image_id'},inplace=True)
df = pd.merge(df, image_train_data_7, on=['image_id','create_date'],how='left', validate='many_to_one')

In [8]:
# 读取ad_account不同窗口期的运营数据
ad_train_data_1 = pd.read_csv('./ad_data_2/train_data_1.csv')
ad_train_data_1['ad_create_role_pay_rate_1'] = ad_train_data_1.apply(lambda x:0 if x.ad_create_role_num_1==0 else 
                                                            x.ad_pay_role_user_num_1 / x.ad_create_role_num_1,axis=1)
ad_train_data_1['ad_create_role_cost_1'] = ad_train_data_1.apply(lambda x:np.inf if x.ad_create_role_num_1==0 else 
                                                            x.ad_amount_1 / x.ad_create_role_num_1,axis=1)

ad_train_data_1['ad_create_role_pay_cost_1'] = ad_train_data_1.apply(lambda x:np.inf if x.ad_pay_role_user_num_1==0 else 
                                                            x.ad_amount_1 / x.ad_pay_role_user_num_1,axis=1)
ad_train_data_1['ad_valid_source_rate_1'] = ad_train_data_1.apply(lambda x:0 if x.ad_source_num_1==0 else 
                                                            x.ad_valid_source_num_1 / x.ad_source_num_1,axis=1)
ad_train_data_1.drop(['data_win_1'],axis=1,inplace=True)
ad_train_data_1['model_run_datetime_1'] = pd.to_datetime(ad_train_data_1['model_run_datetime_1']).dt.date
ad_train_data_1.rename(columns={'model_run_datetime_1':'create_date','ad_account_id_1':'ad_account_id'},inplace=True)
df['create_date'] = pd.to_datetime(df['create_date']).dt.date
df = pd.merge(df, ad_train_data_1, on=['ad_account_id','create_date'],how='left', validate='many_to_one')

ad_train_data_3 = pd.read_csv('./ad_data_2/train_data_3.csv')
ad_train_data_3['ad_create_role_pay_rate_3'] = ad_train_data_3.apply(lambda x:0 if x.ad_create_role_num_3==0 else 
                                                            x.ad_pay_role_user_num_3 / x.ad_create_role_num_3,axis=1)
ad_train_data_3['ad_create_role_cost_3'] = ad_train_data_3.apply(lambda x:np.inf if x.ad_create_role_num_3==0 else 
                                                            x.ad_amount_3 / x.ad_create_role_num_3,axis=1)

ad_train_data_3['ad_create_role_pay_cost_3'] = ad_train_data_3.apply(lambda x:np.inf if x.ad_pay_role_user_num_3==0 else 
                                                            x.ad_amount_3 / x.ad_pay_role_user_num_3,axis=1)
ad_train_data_3['ad_valid_source_rate_3'] = ad_train_data_3.apply(lambda x:0 if x.ad_source_num_3==0 else 
                                                            x.ad_valid_source_num_3 / x.ad_source_num_3,axis=1)
ad_train_data_3.drop(['data_win_3'],axis=1,inplace=True)
ad_train_data_3['model_run_datetime_3'] = pd.to_datetime(ad_train_data_3['model_run_datetime_3']).dt.date
ad_train_data_3.rename(columns={'model_run_datetime_3':'create_date','ad_account_id_3':'ad_account_id'},inplace=True)
df = pd.merge(df, ad_train_data_3, on=['ad_account_id','create_date'],how='left', validate='many_to_one')

ad_train_data_7 = pd.read_csv('./ad_data_2/train_data_7.csv')
ad_train_data_7['ad_create_role_pay_rate_7'] = ad_train_data_7.apply(lambda x:0 if x.ad_create_role_num_7==0 else 
                                                            x.ad_pay_role_user_num_7 / x.ad_create_role_num_7,axis=1)
ad_train_data_7['ad_create_role_cost_7'] = ad_train_data_7.apply(lambda x:np.inf if x.ad_create_role_num_7==0 else 
                                                            x.ad_amount_7 / x.ad_create_role_num_7,axis=1)

ad_train_data_7['ad_create_role_pay_cost_7'] = ad_train_data_7.apply(lambda x:np.inf if x.ad_pay_role_user_num_7==0 else 
                                                            x.ad_amount_7 / x.ad_pay_role_user_num_7,axis=1)
ad_train_data_7['ad_valid_source_rate_7'] = ad_train_data_7.apply(lambda x:0 if x.ad_source_num_7==0 else 
                                                            x.ad_valid_source_num_7 / x.ad_source_num_7,axis=1)
ad_train_data_7.drop(['data_win_7'],axis=1,inplace=True)
ad_train_data_7['model_run_datetime_7'] = pd.to_datetime(ad_train_data_7['model_run_datetime_7']).dt.date
ad_train_data_7.rename(columns={'model_run_datetime_7':'create_date','ad_account_id_7':'ad_account_id'},inplace=True)
df = pd.merge(df, ad_train_data_7, on=['ad_account_id','create_date'],how='left', validate='many_to_one')

In [9]:
df.drop(['interest_words','action_words','budget','bid2','cpa_bid'],axis=1,inplace=True)
# df.drop(['create_role_num','amount','pay_role_user_num','new_role_money'],axis=1,inplace=True)

In [10]:
df.drop(['channel_id','source_id','create_date','tdate'],axis=1,inplace=True)

In [11]:
bucket_col = ['im_amount_1','im_pay_role_user_num_1','im_create_role_num_1','im_new_role_money_1','im_source_num_1','im_valid_source_num_1',
             'im_amount_3','im_pay_role_user_num_3','im_create_role_num_3','im_new_role_money_3','im_source_num_3','im_valid_source_num_3',
             'im_amount_7','im_pay_role_user_num_7','im_create_role_num_7','im_new_role_money_7','im_source_num_7','im_valid_source_num_7',
             'ad_amount_1','ad_pay_role_user_num_1','ad_create_role_num_1','ad_new_role_money_1','ad_source_num_1','ad_valid_source_num_1',
             'ad_amount_3','ad_pay_role_user_num_3','ad_create_role_num_3','ad_new_role_money_3','ad_source_num_3','ad_valid_source_num_3',
             'ad_amount_7','ad_pay_role_user_num_7','ad_create_role_num_7','ad_new_role_money_7','ad_source_num_7','ad_valid_source_num_7']

In [12]:
def optimal_binning_boundary(x: pd.Series, y: pd.Series, nan: float = -999.) -> list:
    '''
        利用决策树获得最优分箱的边界值列表
    '''
    boundary = []  # 待return的分箱边界值列表
    
    x = x.fillna(nan).values  # 填充缺失值
    y = y.values
    from sklearn.tree import DecisionTreeClassifier
    clf = DecisionTreeClassifier(criterion='entropy',    #“信息熵”最小化准则划分
                                 max_leaf_nodes=10,       # 最大叶子节点数
                                 min_samples_leaf=0.05)  # 叶子节点样本数量最小占比

    clf.fit(x.reshape(-1, 1), y)  # 训练决策树
    
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    threshold = clf.tree_.threshold
    
    for i in range(n_nodes):
        if children_left[i] != children_right[i]:  # 获得决策树节点上的划分边界值
            boundary.append(threshold[i])

    boundary.sort()

    min_x = x.min()
    max_x = x.max() + 0.1  # +0.1是为了考虑后续groupby操作时，能包含特征最大值的样本
    boundary = [min_x] + boundary + [max_x]

    return boundary

In [13]:
def bucketed(df, bucket_col):
    for col in bucket_col:
        df[col] = df[col].astype(np.float32)
        bins = optimal_binning_boundary(x=df[col],
                             y=df['label'])
        df[col] = pd.cut(df[col], bins, labels=range(1,len(bins)))
        df[col] = df[col].astype(np.float32)
        df[col] = df[col].fillna(0)
    return df

In [14]:
df = bucketed(df, bucket_col)

In [15]:
df['im_create_role_cost_1'] = pd.cut(df.im_create_role_cost_1, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['im_create_role_pay_cost_1'] = pd.cut(df.im_create_role_pay_cost_1, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])
df['im_create_role_cost_3'] = pd.cut(df.im_create_role_cost_1, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['im_create_role_pay_cost_3'] = pd.cut(df.im_create_role_pay_cost_1, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])
df['im_create_role_cost_7'] = pd.cut(df.im_create_role_cost_1, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['im_create_role_pay_cost_7'] = pd.cut(df.im_create_role_pay_cost_1, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])
df['ad_create_role_cost_1'] = pd.cut(df.im_create_role_cost_1, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['ad_create_role_pay_cost_1'] = pd.cut(df.im_create_role_pay_cost_1, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])
df['ad_create_role_cost_3'] = pd.cut(df.im_create_role_cost_1, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['ad_create_role_pay_cost_3'] = pd.cut(df.im_create_role_pay_cost_1, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])
df['ad_create_role_cost_7'] = pd.cut(df.im_create_role_cost_1, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['ad_create_role_pay_cost_7'] = pd.cut(df.im_create_role_pay_cost_1, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])

In [16]:
df.head()

,ad_account_id,game_id,create_time,inventory_type,schedule_time,delivery_range,flow_control_mode,smart_bid_type,adjust_cpa,hide_if_exists,hide_if_converted,auto_extend_enabled,age,gender,city,location_type,platform,launch_price,retargeting_type,retargeting_tags_include,retargeting_tags_exclude,ac,android_osv,ios_osv,interest_action_mode,interest_categories,action_categories,action_days,action_scene,image_id,label_ids,amount,create_role_num,pay_role_user_num,new_role_money,label,im_amount_1,im_create_role_num_1,im_new_role_money_1,im_pay_role_user_num_1,im_source_num_1,im_valid_source_num_1,im_create_role_pay_rate_1,im_create_role_cost_1,im_create_role_pay_cost_1,im_valid_source_rate_1,im_amount_3,im_create_role_num_3,im_new_role_money_3,im_pay_role_user_num_3,im_source_num_3,im_valid_source_num_3,im_create_role_pay_rate_3,im_create_role_cost_3,im_create_role_pay_cost_3,im_valid_source_rate_3,im_amount_7,im_create_role_num_7,im_new_role_money_7,im_pay_role_user_num_7,im_source_num_7,im_valid_source_num_7,im_create_role_pay_rate_7,im_create_role_cost_7,im_create_role_pay_cost_7,im_valid_source_rate_7,ad_amount_1,ad_create_role_num_1,ad_new_role_money_1,ad_pay_role_user_num_1,ad_source_num_1,ad_valid_source_num_1,ad_create_role_pay_rate_1,ad_create_role_cost_1,ad_create_role_pay_cost_1,ad_valid_source_rate_1,ad_amount_3,ad_create_role_num_3,ad_new_role_money_3,ad_pay_role_user_num_3,ad_source_num_3,ad_valid_source_num_3,ad_create_role_pay_rate_3,ad_create_role_cost_3,ad_create_role_pay_cost_3,ad_valid_source_rate_3,ad_amount_7,ad_create_role_num_7,ad_new_role_money_7,ad_pay_role_user_num_7,ad_source_num_7,ad_valid_source_num_7,ad_create_role_pay_rate_7,ad_create_role_cost_7,ad_create_role_pay_cost_7,ad_valid_source_rate_7
0,4003,1000840,2020-01-19 16:18:31,[INVENTORY_UNION_SLOT],1111111111111111111111111111111111111111111111...,UNION,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19788,NaN,30.88,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
1,4003,1000840,2020-01-19 16:18:31,"[UNION_BOUTIQUE_GAME, INVENTORY_UNION_SLOT]",1111111111111111111111111111111111111111111111...,DEFAULT,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19783,NaN,23.92,5.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN
2,4003,1000840,2020-01-19 16:24:53,"[UNION_BOUTIQUE_GAME, INVENTORY_UNION_SLOT]",1111111111111111111111111111111111111111111111...,DEFAULT,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19636,19,18.36,2.0,0.0,0.0,0,10.0,10.0,10.0,10.0,10.0,9.0,0.029951,2,1,0.453125,10.0,10.0,10.0,10.0,10.0,10.0,0.059982,1,1,0.4875,10.0,10.0,10.0,10.0,9.0,10.0,0.102439,1,1,0.5,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,1,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1,1,NaN
3,4003,1000840,2020-01-20 14:35:23,[INVENTORY_UNION_SLOT],1111111111111111111111111111111111111111111111...,UNION,FLOW_CONTROL_MODE_FAST,SMART_BID_CONSERVATIVE,0.0,0,AD,0,"[AGE_BETWEEN_24_30, AGE_BETWEEN_31_40, AGE_BET...",GENDER_MALE,[],CURRENT,[ANDROID],[],NONE,[],[],[],NONE,NONE,UNLIMITED,[],NaN,NaN,NaN,19791,NaN,1599.31,26.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,

In [17]:
df = df.sort_values(by=['ad_account_id','create_time']).reset_index(drop=True)

In [18]:
df['create_role_cost'] = df['amount']/df['create_role_num']
df['create_role_pay_cost'] = df['amount']/df['pay_role_user_num']
df['roi'] = df['new_role_money']/df['amount']
df['create_role_cost'] = pd.cut(df.create_role_cost, [float('-inf'), 20,40,60,80,100, float('inf')], labels=[1,2,3,4,5,6])
df['create_role_pay_cost'] = pd.cut(df.create_role_pay_cost, [float('-inf'),3000,4000,5000,6000,8000,10000, float('inf')], labels=[1,2,3,4,5,6,7])

In [19]:
df.drop(['create_role_num','amount','pay_role_user_num','new_role_money'],axis=1,inplace=True)

In [20]:
# 对列表内容进行编码降维
def get_mutil_feature(data):
    cols = ['inventory_type','age','city','retargeting_tags_include','retargeting_tags_exclude','ac','interest_categories',
       'action_scene','action_categories']
    for col in cols:
        if col in ['inventory_type','age']:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
        elif col in ['city','retargeting_tags_include','retargeting_tags_exclude','interest_categories','action_categories']:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data[col]=data[col].apply(lambda x:[str(i) for i in x])
            temp = data[col].str.join('|').str.get_dummies()
    #         print(temp.shape[1])
            pca = PCA(n_components=0.9)
            temp = pca.fit_transform(temp.values)
            temp = pd.DataFrame(temp,columns = [col+str(i) for i in range(temp.shape[1])])
    #         print(temp.shape[1])
            data = data.join(temp)
            del temp
            data.drop(col,axis=1,inplace=True)
        else:
            data[col] = data[col].apply(lambda x:x if x==x else [])
            data[col]=data[col].apply(lambda x:[str(i) for i in x])
            data = data.join(data[col].str.join('|').str.get_dummies().add_prefix(col+'_'))
            data.drop(col,axis=1,inplace=True)
    
    gc.collect()
    return data

In [21]:
%%time
df = get_mutil_feature(df)

Wall time: 50.8 s


In [22]:
cats_col = ['ad_account_id','game_id','schedule_time','delivery_range','flow_control_mode','smart_bid_type','hide_if_converted','gender',
           'location_type','platform','launch_price','retargeting_type','android_osv','ios_osv','interest_action_mode',
           'action_days','image_id','label_ids']
for col in cats_col:
    df[col] = df[col].astype(str)
    


In [23]:
test_data = df[df['create_time']>='2020-11-01']
train_data = df[df['create_time']<'2020-11-01']

In [24]:
enc = ce.CatBoostEncoder(cols=cats_col).fit(train_data[cats_col],train_data['label'])
train_data[cats_col] = enc.transform(train_data[cats_col])
test_data[cats_col] = enc.transform(test_data[cats_col])

In [25]:
train_data.head()

,ad_account_id,game_id,create_time,schedule_time,delivery_range,flow_control_mode,smart_bid_type,adjust_cpa,hide_if_exists,hide_if_converted,auto_extend_enabled,gender,location_type,platform,launch_price,retargeting_type,android_osv,ios_osv,interest_action_mode,action_days,image_id,label_ids,label,im_amount_1,im_create_role_num_1,im_new_role_money_1,im_pay_role_user_num_1,im_source_num_1,im_valid_source_num_1,im_create_role_pay_rate_1,im_create_role_cost_1,im_create_role_pay_cost_1,im_valid_source_rate_1,im_amount_3,im_create_role_num_3,im_new_role_money_3,im_pay_role_user_num_3,im_source_num_3,im_valid_source_num_3,im_create_role_pay_rate_3,im_create_role_cost_3,im_create_role_pay_cost_3,im_valid_source_rate_3,im_amount_7,im_create_role_num_7,im_new_role_money_7,im_pay_role_user_num_7,im_source_num_7,im_valid_source_num_7,im_create_role_pay_rate_7,im_create_role_cost_7,im_create_role_pay_cost_7,im_valid_source_rate_7,ad_amount_1,ad_create_role_num_1,ad_new_role_money_1,ad_pay_role_user_num_1,ad_source_num_1,ad_valid_source_num_1,ad_create_role_pay_rate_1,ad_create_role_cost_1,ad_create_role_pay_cost_1,ad_valid_source_rate_1,ad_amount_3,ad_create_role_num_3,ad_new_role_money_3,ad_pay_role_user_num_3,ad_source_num_3,ad_valid_source_num_3,ad_create_role_pay_rate_3,ad_create_role_cost_3,ad_create_role_pay_cost_3,ad_valid_source_rate_3,ad_amount_7,ad_create_role_num_7,ad_new_role_money_7,ad_pay_role_user_num_7,ad_source_num_7,ad_valid_source_num_7,ad_create_role_pay_rate_7,ad_create_role_cost_7,ad_create_role_pay_cost_7,ad_valid_source_rate_7,create_role_cost,create_role_pay_cost,roi,inventory_type_INVENTORY_AWEME_FEED,inventory_type_INVENTORY_AWEME_SEARCH,inventory_type_INVENTORY_FEED,inventory_type_INVENTORY_HOTSOON_FEED,inventory_type_INVENTORY_SEARCH,inventory_type_INVENTORY_UNION_SLOT,inventory_type_INVENTORY_UNION_SPLASH_SLOT,inventory_type_INVENTORY_VIDEO_FEED,inventory_type_UNION_BOUTIQUE_GAME,age_AGE_ABOVE_50,age_AGE_BETWEEN_18_23,age_AGE_BETWEEN_24_30,age_AGE_BETWEEN_31_40,age_AGE_BETWEEN_41_49,city0,city1,city2,retargeting_tags_include0,retargeting_tags_include1,retargeting_tags_include2,retargeting_tags_include3,retargeting_tags_include4,retargeting_tags_include5,retargeting_tags_include6,retargeting_tags_include7,retargeting_tags_include8,retargeting_tags_include9,retargeting_tags_include10,retargeting_tags_include11,retargeting_tags_include12,retargeting_tags_include13,retargeting_tags_include14,retargeting_tags_include15,retargeting_tags_include16,retargeting_tags_include17,retargeting_tags_include18,retargeting_tags_include19,retargeting_tags_include20,retargeting_tags_include21,retargeting_tags_include22,retargeting_tags_include23,retargeting_tags_include24,retargeting_tags_include25,retargeting_tags_include26,retargeting_tags_include27,retargeting_tags_include28,retargeting_tags_include29,retargeting_tags_include30,retargeting_tags_include31,retargeting_tags_include32,retargeting_tags_include33,retargeting_tags_include34,retargeting_tags_include35,retargeting_tags_include36,retargeting_tags_include37,retargeting_tags_include38,retargeting_tags_include39,retargeting_tags_include40,retargeting_tags_include41,retargeting_tags_include42,retargeting_tags_include43,retargeting_tags_include44,retargeting_tags_include45,retargeting_tags_include46,retargeting_tags_include47,retargeting_tags_include48,retargeting_tags_include49,retargeting_tags_include50,retargeting_tags_include51,retargeting_tags_include52,retargeting_tags_include53,retargeting_tags_include54,retargeting_tags_include55,retargeting_tags_include56,retargeting_tags_include57,retargeting_tags_include58,retargeting_tags_include59,retargeting_tags_include60,retargeting_tags_include61,retargeting_tags_include62,retargeting_tags_include63,retargeting_tags_include64,retargeting_tags_include65,retargeting_tags_include66,retargeting_tags_include67,retargeting_tags_include68,retargeting_tags_include69,retargeting_tags_include70,retargeting_tags_include71,retargeting_

In [26]:
for col in train_data.columns:
    if col not in  ['create_time','label']:
        train_data[col] = train_data[col].astype(np.float32)
        ss = StandardScaler()
        train_data[col] = ss.fit_transform(train_data[col].values.reshape(-1,1))
for col in test_data.columns:
    if col not in  ['create_time','label']:
        test_data[col] = test_data[col].astype(np.float32)
        ss = StandardScaler()
        test_data[col] = ss.fit_transform(test_data[col].values.reshape(-1,1))

In [27]:
df = pd.concat([train_data,test_data],axis=0)

In [28]:
df = df.sort_values(by=['ad_account_id','create_time']).reset_index(drop=True)

In [29]:
df.head()

,ad_account_id,game_id,create_time,schedule_time,delivery_range,flow_control_mode,smart_bid_type,adjust_cpa,hide_if_exists,hide_if_converted,auto_extend_enabled,gender,location_type,platform,launch_price,retargeting_type,android_osv,ios_osv,interest_action_mode,action_days,image_id,label_ids,label,im_amount_1,im_create_role_num_1,im_new_role_money_1,im_pay_role_user_num_1,im_source_num_1,im_valid_source_num_1,im_create_role_pay_rate_1,im_create_role_cost_1,im_create_role_pay_cost_1,im_valid_source_rate_1,im_amount_3,im_create_role_num_3,im_new_role_money_3,im_pay_role_user_num_3,im_source_num_3,im_valid_source_num_3,im_create_role_pay_rate_3,im_create_role_cost_3,im_create_role_pay_cost_3,im_valid_source_rate_3,im_amount_7,im_create_role_num_7,im_new_role_money_7,im_pay_role_user_num_7,im_source_num_7,im_valid_source_num_7,im_create_role_pay_rate_7,im_create_role_cost_7,im_create_role_pay_cost_7,im_valid_source_rate_7,ad_amount_1,ad_create_role_num_1,ad_new_role_money_1,ad_pay_role_user_num_1,ad_source_num_1,ad_valid_source_num_1,ad_create_role_pay_rate_1,ad_create_role_cost_1,ad_create_role_pay_cost_1,ad_valid_source_rate_1,ad_amount_3,ad_create_role_num_3,ad_new_role_money_3,ad_pay_role_user_num_3,ad_source_num_3,ad_valid_source_num_3,ad_create_role_pay_rate_3,ad_create_role_cost_3,ad_create_role_pay_cost_3,ad_valid_source_rate_3,ad_amount_7,ad_create_role_num_7,ad_new_role_money_7,ad_pay_role_user_num_7,ad_source_num_7,ad_valid_source_num_7,ad_create_role_pay_rate_7,ad_create_role_cost_7,ad_create_role_pay_cost_7,ad_valid_source_rate_7,create_role_cost,create_role_pay_cost,roi,inventory_type_INVENTORY_AWEME_FEED,inventory_type_INVENTORY_AWEME_SEARCH,inventory_type_INVENTORY_FEED,inventory_type_INVENTORY_HOTSOON_FEED,inventory_type_INVENTORY_SEARCH,inventory_type_INVENTORY_UNION_SLOT,inventory_type_INVENTORY_UNION_SPLASH_SLOT,inventory_type_INVENTORY_VIDEO_FEED,inventory_type_UNION_BOUTIQUE_GAME,age_AGE_ABOVE_50,age_AGE_BETWEEN_18_23,age_AGE_BETWEEN_24_30,age_AGE_BETWEEN_31_40,age_AGE_BETWEEN_41_49,city0,city1,city2,retargeting_tags_include0,retargeting_tags_include1,retargeting_tags_include2,retargeting_tags_include3,retargeting_tags_include4,retargeting_tags_include5,retargeting_tags_include6,retargeting_tags_include7,retargeting_tags_include8,retargeting_tags_include9,retargeting_tags_include10,retargeting_tags_include11,retargeting_tags_include12,retargeting_tags_include13,retargeting_tags_include14,retargeting_tags_include15,retargeting_tags_include16,retargeting_tags_include17,retargeting_tags_include18,retargeting_tags_include19,retargeting_tags_include20,retargeting_tags_include21,retargeting_tags_include22,retargeting_tags_include23,retargeting_tags_include24,retargeting_tags_include25,retargeting_tags_include26,retargeting_tags_include27,retargeting_tags_include28,retargeting_tags_include29,retargeting_tags_include30,retargeting_tags_include31,retargeting_tags_include32,retargeting_tags_include33,retargeting_tags_include34,retargeting_tags_include35,retargeting_tags_include36,retargeting_tags_include37,retargeting_tags_include38,retargeting_tags_include39,retargeting_tags_include40,retargeting_tags_include41,retargeting_tags_include42,retargeting_tags_include43,retargeting_tags_include44,retargeting_tags_include45,retargeting_tags_include46,retargeting_tags_include47,retargeting_tags_include48,retargeting_tags_include49,retargeting_tags_include50,retargeting_tags_include51,retargeting_tags_include52,retargeting_tags_include53,retargeting_tags_include54,retargeting_tags_include55,retargeting_tags_include56,retargeting_tags_include57,retargeting_tags_include58,retargeting_tags_include59,retargeting_tags_include60,retargeting_tags_include61,retargeting_tags_include62,retargeting_tags_include63,retargeting_tags_include64,retargeting_tags_include65,retargeting_tags_include66,retargeting_tags_include67,retargeting_tags_include68,retargeting_tags_include69,retargeting_tags_include70,retargeting_tags_include71,retargeting_

In [30]:
# 时间序列数据
seq_df = df[['ad_account_id','game_id','create_time','platform','image_id', 'label_ids',
             'ad_amount_3','ad_pay_role_user_num_3','ad_create_role_num_3','ad_new_role_money_3',
             'ad_source_num_3','ad_valid_source_num_3','ad_create_role_cost_3','ad_create_role_pay_cost_3',
             'im_amount_3','im_pay_role_user_num_3','im_create_role_num_3','im_new_role_money_3',
             'im_source_num_3','im_valid_source_num_3','im_create_role_cost_3','im_create_role_pay_cost_3',
             'create_role_cost','create_role_pay_cost', 'roi']]

In [31]:
def extract_seq(df, maxlen=10):
    values=[]
    for i in range(0,df.shape[0]):
#         if i <100:
        value = df[(df['ad_account_id']==df.iloc[i]['ad_account_id']) & 
           (df['create_time']>=df.iloc[i]['create_time']-pd.DateOffset(days=7)) &
          (df['create_time']<=df.iloc[i]['create_time']-pd.DateOffset(hours=3))].drop(['ad_account_id','create_time'],axis=1).values.tolist()
#             print(shape(value))
    #         print(value)
        if len(value) == 0:
            value = np.zeros((maxlen,df.shape[1]-2)).tolist()
#                 print(shape(value))
        if len(value)<=maxlen:
            value = np.pad(value,((maxlen-len(value),0),(0,0))).tolist()
#                 print(shape(value))
        else:
            value = value[len(value)-maxlen:len(value)]
#                 print(shape(value))
        values.append(value)
#         print(shape(values))    
    return values

In [32]:
%%time
seq_values = extract_seq(seq_df)

Wall time: 58.6 s


In [33]:
np.shape(seq_values)

(28562, 10, 23)

In [34]:
df.head()

,ad_account_id,game_id,create_time,schedule_time,delivery_range,flow_control_mode,smart_bid_type,adjust_cpa,hide_if_exists,hide_if_converted,auto_extend_enabled,gender,location_type,platform,launch_price,retargeting_type,android_osv,ios_osv,interest_action_mode,action_days,image_id,label_ids,label,im_amount_1,im_create_role_num_1,im_new_role_money_1,im_pay_role_user_num_1,im_source_num_1,im_valid_source_num_1,im_create_role_pay_rate_1,im_create_role_cost_1,im_create_role_pay_cost_1,im_valid_source_rate_1,im_amount_3,im_create_role_num_3,im_new_role_money_3,im_pay_role_user_num_3,im_source_num_3,im_valid_source_num_3,im_create_role_pay_rate_3,im_create_role_cost_3,im_create_role_pay_cost_3,im_valid_source_rate_3,im_amount_7,im_create_role_num_7,im_new_role_money_7,im_pay_role_user_num_7,im_source_num_7,im_valid_source_num_7,im_create_role_pay_rate_7,im_create_role_cost_7,im_create_role_pay_cost_7,im_valid_source_rate_7,ad_amount_1,ad_create_role_num_1,ad_new_role_money_1,ad_pay_role_user_num_1,ad_source_num_1,ad_valid_source_num_1,ad_create_role_pay_rate_1,ad_create_role_cost_1,ad_create_role_pay_cost_1,ad_valid_source_rate_1,ad_amount_3,ad_create_role_num_3,ad_new_role_money_3,ad_pay_role_user_num_3,ad_source_num_3,ad_valid_source_num_3,ad_create_role_pay_rate_3,ad_create_role_cost_3,ad_create_role_pay_cost_3,ad_valid_source_rate_3,ad_amount_7,ad_create_role_num_7,ad_new_role_money_7,ad_pay_role_user_num_7,ad_source_num_7,ad_valid_source_num_7,ad_create_role_pay_rate_7,ad_create_role_cost_7,ad_create_role_pay_cost_7,ad_valid_source_rate_7,create_role_cost,create_role_pay_cost,roi,inventory_type_INVENTORY_AWEME_FEED,inventory_type_INVENTORY_AWEME_SEARCH,inventory_type_INVENTORY_FEED,inventory_type_INVENTORY_HOTSOON_FEED,inventory_type_INVENTORY_SEARCH,inventory_type_INVENTORY_UNION_SLOT,inventory_type_INVENTORY_UNION_SPLASH_SLOT,inventory_type_INVENTORY_VIDEO_FEED,inventory_type_UNION_BOUTIQUE_GAME,age_AGE_ABOVE_50,age_AGE_BETWEEN_18_23,age_AGE_BETWEEN_24_30,age_AGE_BETWEEN_31_40,age_AGE_BETWEEN_41_49,city0,city1,city2,retargeting_tags_include0,retargeting_tags_include1,retargeting_tags_include2,retargeting_tags_include3,retargeting_tags_include4,retargeting_tags_include5,retargeting_tags_include6,retargeting_tags_include7,retargeting_tags_include8,retargeting_tags_include9,retargeting_tags_include10,retargeting_tags_include11,retargeting_tags_include12,retargeting_tags_include13,retargeting_tags_include14,retargeting_tags_include15,retargeting_tags_include16,retargeting_tags_include17,retargeting_tags_include18,retargeting_tags_include19,retargeting_tags_include20,retargeting_tags_include21,retargeting_tags_include22,retargeting_tags_include23,retargeting_tags_include24,retargeting_tags_include25,retargeting_tags_include26,retargeting_tags_include27,retargeting_tags_include28,retargeting_tags_include29,retargeting_tags_include30,retargeting_tags_include31,retargeting_tags_include32,retargeting_tags_include33,retargeting_tags_include34,retargeting_tags_include35,retargeting_tags_include36,retargeting_tags_include37,retargeting_tags_include38,retargeting_tags_include39,retargeting_tags_include40,retargeting_tags_include41,retargeting_tags_include42,retargeting_tags_include43,retargeting_tags_include44,retargeting_tags_include45,retargeting_tags_include46,retargeting_tags_include47,retargeting_tags_include48,retargeting_tags_include49,retargeting_tags_include50,retargeting_tags_include51,retargeting_tags_include52,retargeting_tags_include53,retargeting_tags_include54,retargeting_tags_include55,retargeting_tags_include56,retargeting_tags_include57,retargeting_tags_include58,retargeting_tags_include59,retargeting_tags_include60,retargeting_tags_include61,retargeting_tags_include62,retargeting_tags_include63,retargeting_tags_include64,retargeting_tags_include65,retargeting_tags_include66,retargeting_tags_include67,retargeting_tags_include68,retargeting_tags_include69,retargeting_tags_include70,retargeting_tags_include71,retargeting_

In [294]:
ss = x_train.values

In [291]:
x_train.shape

(21819, 315)

In [295]:
ss.reshape((21819,1,315))

array([[[-1.4184445 , -1.4427575 ,  0.3447388 , ..., -0.03898377,
          0.02896921,  0.01880279]],

       [[-1.4184445 , -1.4427575 ,  0.3447388 , ..., -0.03898377,
          0.02896921,  0.01880279]],

       [[-1.4184445 , -1.4427575 ,  0.3447388 , ..., -0.03898377,
          0.02896921,  0.01880279]],

       ...,

       [[ 5.3997116 , -1.4427575 , -2.9922156 , ..., -0.03898377,
          0.02896921,  0.01880279]],

       [[ 5.3997116 , -1.4427575 , -0.04449505, ..., -0.03898377,
          0.02896921,  0.01880279]],

       [[ 5.3997116 , -1.4427575 , -2.9922156 , ..., -0.03898377,
          0.02896921,  0.01880279]]], dtype=float32)

In [ ]:
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [35]:
test_data = df[df['create_time']>='2020-11-01']
train_data = df[df['create_time']<'2020-11-01']

In [36]:
train_data.drop(['create_time','create_role_cost','create_role_pay_cost', 'roi'],axis=1,inplace=True)
test_data.drop(['create_time','create_role_cost','create_role_pay_cost', 'roi'],axis=1,inplace=True)

In [37]:
train_data.fillna(0,inplace=True)
test_data.fillna(0,inplace=True)

In [38]:
seq_values = np.array(seq_values)
seq_values[np.isnan(seq_values)] = 0

In [39]:
train_seq = seq_values[train_data.index.tolist()]
test_seq = seq_values[test_data.index.tolist()]

In [40]:
train_data.shape

(21819, 313)

In [44]:
feature_data.shape

(21819, 312)

In [45]:
x_train = train_data.drop('label',axis=1)
y_train = train_data['label']

In [46]:
# 载入输入数据
seq_data = train_seq
feature_data = train_data.drop('label',axis=1).values

# 标签
label = train_data['label'].values

In [47]:
np.shape(train_seq)

(21819, 10, 23)

In [48]:
feature_data.shape

(21819, 312)

In [49]:
len(label)

21819

In [50]:
tf.keras.backend.clear_session()

# 输入

# seq_input = keras.Input(shape=(seq_data.shape[1],seq_data.shape[2],), name='seq')
feature_input = tf.keras.Input(shape=(x_train.shape[1],), name='feature')

# 特征提取层
# seq_feat = layers.LSTM(128)(seq_input)
# h2 = layers.LSTM(64,return_sequences=True)(h1)
#  = layers.LSTM(32)(h1)


# feature_input = layers.Embedding(64)(feature_input)
# features = layers.concatenate([seq_feat,feature_input])

d1 = layers.BatchNormalization()(feature_input)
d1 = layers.Dropout(rate=0.4)(d1)

h1 = layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))(d1)

h1 = layers.BatchNormalization()(h1)
h1 = layers.Dropout(rate=0.3)(h1)

h2 = layers.Dense(128, activation='relu')(h1)
h2 = layers.BatchNormalization()(h2)
h2 = layers.Dropout(rate=0.3)(h2)

h3 = layers.Dense(64, activation='relu')(h2)
h3 = layers.BatchNormalization()(h3)
h3 = layers.Dropout(rate=0.3)(h3)

h4 = layers.Dense(32, activation='relu')(h3)
h4 = layers.BatchNormalization()(h4)
h4 = layers.Dropout(rate=0.3)(h4)

h5 = layers.Dense(16, activation='relu')(h4)
h5 = layers.BatchNormalization()(h5)
h5 = layers.Dropout(rate=0.3)(h5)


outputs = layers.Dense(1, activation='sigmoid',name='priority')(h5)

# 构建模型
model = keras.Model(inputs=feature_input,
                    outputs=outputs)
# model.summary()                            
# 分类层
# priority_pred = layers.Dense(1, activation='sigmoid', name='priority')(features)

In [51]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['AUC'])

In [52]:
history = model.fit(x_train,y_train,
                    batch_size= 64,
                    validation_split=0.3,
                    verbose=1,
                    epochs=40)

Epoch 1/40
239/239 [==============================] - 1s 5ms/step - loss: 2.2964 - auc: 0.5188 - val_loss: 1.4615 - val_auc: 0.5250
Epoch 2/40
239/239 [==============================] - 1s 3ms/step - loss: 0.9266 - auc: 0.5259 - val_loss: 0.8901 - val_auc: 0.6092
Epoch 3/40
239/239 [==============================] - 1s 3ms/step - loss: 0.5457 - auc: 0.5452 - val_loss: 0.7343 - val_auc: 0.6255
Epoch 4/40
239/239 [==============================] - 1s 3ms/step - loss: 0.4284 - auc: 0.5887 - val_loss: 0.6566 - val_auc: 0.6859
Epoch 5/40
239/239 [==============================] - 1s 3ms/step - loss: 0.3802 - auc: 0.6279 - val_loss: 0.6131 - val_auc: 0.7079
Epoch 6/40
239/239 [==============================] - 1s 3ms/step - loss: 0.3503 - auc: 0.6433 - val_loss: 0.5764 - val_auc: 0.7127
Epoch 7/40
239/239 [==============================] - 1s 3ms/step - loss: 0.3281 - auc: 0.6687 - val_loss: 0.5621 - val_auc: 0.7192
Epoch 8/40
239/239 [==============================] - 1s 3ms/step - loss: 0.

In [64]:
tf.keras.backend.clear_session()

# 输入

seq_input = keras.Input(shape=(seq_data.shape[1],seq_data.shape[2],), name='seq')
feature_input = keras.Input(shape=(feature_data.shape[1],), name='feature')

# 特征提取层
seq_feat = layers.LSTM(128)(seq_input)
# h2 = layers.LSTM(64,return_sequences=True)(h1)
#  = layers.LSTM(32)(h1)


# feature_input = layers.Embedding(64)(feature_input)
features = layers.concatenate([seq_feat,feature_input])

d1 = layers.BatchNormalization()(features)
d1 = layers.Dropout(rate=0.2)(d1)

h1 = layers.Dense(256, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.01))(d1)

h1 = layers.BatchNormalization()(h1)
h1 = layers.Dropout(rate=0.1)(h1)

h2 = layers.Dense(128, activation='relu')(h1)
h2 = layers.BatchNormalization()(h2)
h2 = layers.Dropout(rate=0.05)(h2)

h3 = layers.Dense(64, activation='relu')(h2)
h3 = layers.BatchNormalization()(h3)
h3 = layers.Dropout(rate=0.05)(h3)

h4 = layers.Dense(32, activation='relu')(h3)
h4 = layers.BatchNormalization()(h4)
h4 = layers.Dropout(rate=0.05)(h4)

h5 = layers.Dense(16, activation='relu')(h4)
h5 = layers.BatchNormalization()(h5)
h5 = layers.Dropout(rate=0.05)(h5)


outputs = layers.Dense(1, activation='sigmoid',name='priority')(h5)

# 构建模型
model = keras.Model(inputs=[seq_input, feature_input],
                    outputs=outputs)
# model.summary()                            
# 分类层
# priority_pred = layers.Dense(1, activation='sigmoid', name='priority')(features)

In [65]:
model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['AUC'])

In [66]:
history = model.fit(
    {'seq': seq_data, 'feature':feature_data},
    {'priority':label},
    batch_size=32,
    epochs=50,
    validation_split=0.3,
    verbose=1)

Epoch 1/50
478/478 [==============================] - 4s 8ms/step - loss: 1.4825 - auc: 0.5328 - val_loss: 0.7111 - val_auc: 0.6191
Epoch 2/50
478/478 [==============================] - 3s 7ms/step - loss: 0.3687 - auc: 0.6275 - val_loss: 0.5719 - val_auc: 0.6853
Epoch 3/50
478/478 [==============================] - 4s 7ms/step - loss: 0.3170 - auc: 0.6701 - val_loss: 0.5529 - val_auc: 0.7136
Epoch 4/50
478/478 [==============================] - 4s 8ms/step - loss: 0.2928 - auc: 0.6964 - val_loss: 0.5293 - val_auc: 0.7209
Epoch 5/50
478/478 [==============================] - 4s 8ms/step - loss: 0.2886 - auc: 0.7022 - val_loss: 0.5141 - val_auc: 0.7290
Epoch 6/50
478/478 [==============================] - 4s 7ms/step - loss: 0.2847 - auc: 0.7048 - val_loss: 0.5530 - val_auc: 0.6931
Epoch 7/50
478/478 [==============================] - 4s 7ms/step - loss: 0.2713 - auc: 0.7149 - val_loss: 0.5541 - val_auc: 0.6626
Epoch 8/50
478/478 [==============================] - 4s 7ms/step - loss: 0.

In [ ]:
seq_data = train_seq
feature_data = train_data.drop('label',axis=1).values

# 标签
label = train_data['label'].values

In [67]:
# 测试数据
seq_data_test = test_seq
feature_data_test = test_data.drop('label',axis=1).values
label_test = test_data['label'].values

In [68]:
test_loss, test_acc = model.evaluate({'seq': seq_data_test, 'feature':feature_data_test},{'priority':label_test})

211/211 [==============================] - 0s 2ms/step - loss: 0.4316 - auc: 0.5844


In [55]:
predict_y = model.predict({'seq': seq_data_test, 'feature':feature_data_test})

In [59]:
sum(list(map(lambda x:1 if x>=0.16 else 0,predict_y)))

806

In [60]:
sum(label_test)

723

In [61]:
from sklearn.metrics import f1_score,classification_report

In [62]:
predict_y = list(map(lambda x:1 if x>=0.25 else 0,predict_y))

In [63]:
print(classification_report(label_test, predict_y))

              precision    recall  f1-score   support

           0       0.90      0.98      0.94      6020
           1       0.24      0.05      0.09       723

    accuracy                           0.88      6743
   macro avg       0.57      0.52      0.51      6743
weighted avg       0.83      0.88      0.84      6743

